# Variability Tool

In [ ]:
import inro.modeller as _m
import inro.emme.desktop as _d
import traceback as _traceback

import os
import re
import copy
import csv as csv

import sqlite3
import pandas as pd
import numpy as np

## Basic Setup

In [ ]:
util = _m.Modeller().tool("translink.util")
data_export = _m.Modeller().tool("translink.RTM3.stage4.dataexport") 
copy_scenario = _m.Modeller().tool("inro.emme.data.scenario.copy_scenario")
create_scenarios = _m.Modeller().tool("translink.RTM3.stage0.create_scenarios")
auto_assign = _m.Modeller().tool("translink.RTM3.stage3.autoassignment")


In [ ]:
dt = _d.app.connect()
de = dt.data_explorer()
db = de.active_database()
ebs = de.databases()
ebs = de.databases()

In [ ]:
eb = _m.Modeller().emmebank
# scenario variables
base_scenario = eb.scenario(2000)
am_scenid = int(eb.matrix("msAmScen").data)
md_scenid = int(eb.matrix("msMdScen").data)
pm_scenid = int(eb.matrix("msPmScen").data)
# am_scen = eb.scenario(int(eb.matrix("msAmScen").data)) # not clear that these are needed
# md_scen = eb.scenario(int(eb.matrix("msMdScen").data))
# pm_scen = eb.scenario(int(eb.matrix("msPmScen").data))

# factor variables
days = 10
mean_days_share = 0.5
daily_range = 0.10
sd = 0.05
noTAZ = len(util.get_matrix_numpy(eb, "zoneindex"))

## Functions

In [ ]:
def generate_sample_mean(i, days, daily_range, mean_days_share):
    if mean_days_share == 1.0:
        sample_mean = 1.0
    
    elif mean_days_share == 0.0 and i <= days * 0.5:
        sample_mean = round(1.0 - daily_range + (((i-1) - days * mean_days_share) * ((2 * daily_range) / (days * (1 - mean_days_share)))), 2)

    elif mean_days_share == 0.0 and i > days * 0.5:
        sample_mean = round(1.0 - daily_range + ((i - days * mean_days_share) * ((2 * daily_range) / (days * (1 - mean_days_share)))), 2)
    
    elif i < days * mean_days_share:
        sample_mean = 1.0
    
    else:
        sample_mean = round(1.0 - daily_range + ((i - days * mean_days_share) * ((2 * daily_range) / (days * (1 - mean_days_share)))), 2)
    return sample_mean

In [ ]:
def factor_matrix(matrix, sample_mean, sd, noTAZ):
    facmat = np.random.normal(loc=sample_mean, scale=sd, size = [noTAZ,noTAZ])
    floormat = np.empty((noTAZ,noTAZ))
    floormat.fill(0.01)
    facmat = np.maximum(facmat, floormat)
    outmat = matrix * facmat
    return outmat

In [ ]:
def matrix_batchins(eb):
    util = _m.Modeller().tool("translink.util")
    # Initialize Copies of original demand matrics
    util.initmat(eb, "mf1300", "SOV_drvtrp_VOT_1_AmC", "SOV drv-trips VOT 1 AM C", 0)
    util.initmat(eb, "mf1301", "SOV_drvtrp_VOT_2_AmC", "SOV drv-trips VOT 2 AM C", 0)
    util.initmat(eb, "mf1302", "SOV_drvtrp_VOT_3_AmC", "SOV drv-trips VOT 3 AM C", 0)
    util.initmat(eb, "mf1303", "SOV_drvtrp_VOT_4_AmC", "SOV drv-trips VOT 4 AM C", 0)
    util.initmat(eb, "mf1306", "HOV_drvtrp_VOT_1_AmC", "HOV drv-trips VOT 1 AM C", 0)
    util.initmat(eb, "mf1307", "HOV_drvtrp_VOT_2_AmC", "HOV drv-trips VOT 2 AM C", 0)
    util.initmat(eb, "mf1308", "HOV_drvtrp_VOT_3_AmC", "HOV drv-trips VOT 3 AM C", 0)
    util.initmat(eb, "mf1309", "HOV_drvtrp_VOT_4_AmC", "HOV drv-trips VOT 4 AM C", 0)
    util.initmat(eb, "mf1320", "SOV_drvtrp_VOT_1_MdC", "SOV drv-trips VOT 1 MD C", 0)
    util.initmat(eb, "mf1321", "SOV_drvtrp_VOT_2_MdC", "SOV drv-trips VOT 2 MD C", 0)
    util.initmat(eb, "mf1322", "SOV_drvtrp_VOT_3_MdC", "SOV drv-trips VOT 3 MD C", 0)
    util.initmat(eb, "mf1323", "SOV_drvtrp_VOT_4_MdC", "SOV drv-trips VOT 4 MD C", 0)
    util.initmat(eb, "mf1326", "HOV_drvtrp_VOT_1_MdC", "HOV drv-trips VOT 1 MD C", 0)
    util.initmat(eb, "mf1327", "HOV_drvtrp_VOT_2_MdC", "HOV drv-trips VOT 2 MD C", 0)
    util.initmat(eb, "mf1328", "HOV_drvtrp_VOT_3_MdC", "HOV drv-trips VOT 3 MD C", 0)
    util.initmat(eb, "mf1329", "HOV_drvtrp_VOT_4_MdC", "HOV drv-trips VOT 4 MD C", 0)
    util.initmat(eb, "mf1340", "SOV_drvtrp_VOT_1_PmC", "SOV drv-trips VOT 1 PM C", 0)
    util.initmat(eb, "mf1341", "SOV_drvtrp_VOT_2_PmC", "SOV drv-trips VOT 2 PM C", 0)
    util.initmat(eb, "mf1342", "SOV_drvtrp_VOT_3_PmC", "SOV drv-trips VOT 3 PM C", 0)
    util.initmat(eb, "mf1343", "SOV_drvtrp_VOT_4_PmC", "SOV drv-trips VOT 4 PM C", 0)
    util.initmat(eb, "mf1346", "HOV_drvtrp_VOT_1_PmC", "HOV drv-trips VOT 1 PM C", 0)
    util.initmat(eb, "mf1347", "HOV_drvtrp_VOT_2_PmC", "HOV drv-trips VOT 2 PM C", 0)
    util.initmat(eb, "mf1348", "HOV_drvtrp_VOT_3_PmC", "HOV drv-trips VOT 3 PM C", 0)
    util.initmat(eb, "mf1349", "HOV_drvtrp_VOT_4_PmC", "HOV drv-trips VOT 4 PM C", 0)

    util.initmat(eb, "mf1312", "lgvPceAmC", "light trucks PCE AM C", 0)
    util.initmat(eb, "mf1313", "hgvPceAmC", "heavy trucks PCE AM C", 0)
    util.initmat(eb, "mf1332", "lgvPceMdC", "light trucks PCE MD C", 0)
    util.initmat(eb, "mf1333", "hgvPceMdC", "heavy trucks PCE MD C", 0)
    util.initmat(eb, "mf1352", "lgvPcePmC", "light trucks PCE PM C", 0)
    util.initmat(eb, "mf1353", "hgvPcePmC", "heavy trucks PCE PM C", 0)


In [ ]:
amSovDemand1 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Am")
amSovDemand2 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Am")
amSovDemand3 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Am")
amSovDemand4 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Am")

mdSovDemand1 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Md")
mdSovDemand2 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Md")
mdSovDemand3 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Md")
mdSovDemand4 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Md")

pmSovDemand1 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Pm")
pmSovDemand2 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Pm")
pmSovDemand3 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Pm")
pmSovDemand4 = util.get_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Pm")

amHovDemand1 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Am")
amHovDemand2 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Am")
amHovDemand3 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Am")
amHovDemand4 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Am")

mdHovDemand1 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Md")
mdHovDemand2 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Md")
mdHovDemand3 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Md")
mdHovDemand4 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Md")

pmHovDemand1 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Pm")
pmHovDemand2 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Pm")
pmHovDemand3 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Pm")
pmHovDemand4 = util.get_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Pm")

amLGVDemand1 = util.get_matrix_numpy(eb, "mflgvPceAm")
amHGVDemand1 = util.get_matrix_numpy(eb, "mfhgvPceAm")
mdLGVDemand1 = util.get_matrix_numpy(eb, "mflgvPceMd")
mdHGVDemand1 = util.get_matrix_numpy(eb, "mfhgvPceMd")
pmLGVDemand1 = util.get_matrix_numpy(eb, "mflgvPcePm")
pmHGVDemand1 = util.get_matrix_numpy(eb, "mfhgvPcePm")

In [ ]:
matrix_batchins(eb)

In [ ]:
# save copies of demand matrices in case run crashes with factored matrices in place
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_AmC", amSovDemand1)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_AmC", amSovDemand2)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_AmC", amSovDemand3)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_AmC", amSovDemand4)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_MdC", mdSovDemand1)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_MdC", mdSovDemand2)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_MdC", mdSovDemand3)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_MdC", mdSovDemand4)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_PmC", pmSovDemand1)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_PmC", pmSovDemand2)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_PmC", pmSovDemand3)
util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_PmC", pmSovDemand4)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_AmC", amHovDemand1)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_AmC", amHovDemand2)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_AmC", amHovDemand3)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_AmC", amHovDemand4)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_MdC", mdHovDemand1)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_MdC", mdHovDemand2)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_MdC", mdHovDemand3)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_MdC", mdHovDemand4)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_PmC", pmHovDemand1)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_PmC", pmHovDemand2)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_PmC", pmHovDemand3)
util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_PmC", pmHovDemand4)
util.set_matrix_numpy(eb, "mflgvPceAmC", amLGVDemand1)
util.set_matrix_numpy(eb, "mfhgvPceAmC", amHGVDemand1)
util.set_matrix_numpy(eb, "mflgvPceMdC", mdLGVDemand1)
util.set_matrix_numpy(eb, "mfhgvPceMdC", mdHGVDemand1)
util.set_matrix_numpy(eb, "mflgvPcePmC", pmLGVDemand1)
util.set_matrix_numpy(eb, "mfhgvPcePmC", pmHGVDemand1)

## The loop!

In [ ]:
for i in range(1, days+1):
    
    # Copy to new AM scenarios
    am_scenid_i = am_scenid + 100 + i
    copy_scenario(from_scenario=base_scenario,
                  scenario_id=am_scenid_i,
                  scenario_title="{} AM day {}".format(base_scenario.title, i),
                  overwrite=True)
    amscen_i = eb.scenario(am_scenid_i)

    create_scenarios.attribute_code(amscen_i, "@lanesam", "@vdfam", "@tpfam", "@hdwyam", "@tollam")

    # Copy to new MD Scenarios
    md_scenid_i = md_scenid + 100 + i
    copy_scenario(from_scenario=base_scenario,
                  scenario_id=md_scenid_i,
                  scenario_title="{} MD day {}".format(base_scenario.title, i),
                  overwrite=True)
    mdscen_i = eb.scenario(md_scenid_i)

    create_scenarios.attribute_code(mdscen_i, "@lanesmd", "@vdfmd", "@tpfmd", "@hdwymd", "@tollmd")

    # Copy to new pm Scenarios
    pm_scenid_i = pm_scenid + 100 + i
    copy_scenario(from_scenario=base_scenario,
                  scenario_id=pm_scenid_i,
                  scenario_title="{} PM day {}".format(base_scenario.title, i),
                  overwrite=True)
    pmscen_i = eb.scenario(pm_scenid_i)

    create_scenarios.attribute_code(pmscen_i, "@lanespm", "@vdfpm", "@tpfpm", "@hdwypm", "@tollpm")

    create_scenarios.overide_network(amscen_i, mdscen_i, pmscen_i) # should probably just import from somewhere rather than copy
    
    # factor the matrices
    sample_mean = generate_sample_mean(i, days, daily_range, mean_days_share)
    amSovDemand1_fac = factor_matrix(amSovDemand1, sample_mean, sd, noTAZ)
    amSovDemand2_fac = factor_matrix(amSovDemand2, sample_mean, sd, noTAZ)
    amSovDemand3_fac = factor_matrix(amSovDemand3, sample_mean, sd, noTAZ)
    amSovDemand4_fac = factor_matrix(amSovDemand4, sample_mean, sd, noTAZ)
    mdSovDemand1_fac = factor_matrix(mdSovDemand1, sample_mean, sd, noTAZ)
    mdSovDemand2_fac = factor_matrix(mdSovDemand2, sample_mean, sd, noTAZ)
    mdSovDemand3_fac = factor_matrix(mdSovDemand3, sample_mean, sd, noTAZ)
    mdSovDemand4_fac = factor_matrix(mdSovDemand4, sample_mean, sd, noTAZ)
    pmSovDemand1_fac = factor_matrix(pmSovDemand1, sample_mean, sd, noTAZ)
    pmSovDemand2_fac = factor_matrix(pmSovDemand2, sample_mean, sd, noTAZ)
    pmSovDemand3_fac = factor_matrix(pmSovDemand3, sample_mean, sd, noTAZ)
    pmSovDemand4_fac = factor_matrix(pmSovDemand4, sample_mean, sd, noTAZ)
    amHovDemand1_fac = factor_matrix(amHovDemand1, sample_mean, sd, noTAZ)
    amHovDemand2_fac = factor_matrix(amHovDemand2, sample_mean, sd, noTAZ)
    amHovDemand3_fac = factor_matrix(amHovDemand3, sample_mean, sd, noTAZ)
    amHovDemand4_fac = factor_matrix(amHovDemand4, sample_mean, sd, noTAZ)
    mdHovDemand1_fac = factor_matrix(mdHovDemand1, sample_mean, sd, noTAZ)
    mdHovDemand2_fac = factor_matrix(mdHovDemand2, sample_mean, sd, noTAZ)
    mdHovDemand3_fac = factor_matrix(mdHovDemand3, sample_mean, sd, noTAZ)
    mdHovDemand4_fac = factor_matrix(mdHovDemand4, sample_mean, sd, noTAZ)
    pmHovDemand1_fac = factor_matrix(pmHovDemand1, sample_mean, sd, noTAZ)
    pmHovDemand2_fac = factor_matrix(pmHovDemand2, sample_mean, sd, noTAZ)
    pmHovDemand3_fac = factor_matrix(pmHovDemand3, sample_mean, sd, noTAZ)
    pmHovDemand4_fac = factor_matrix(pmHovDemand4, sample_mean, sd, noTAZ)
    amLGVDemand1_fac = factor_matrix(amLGVDemand1, sample_mean, sd, noTAZ)
    amHGVDemand1_fac = factor_matrix(amHGVDemand1, sample_mean, sd, noTAZ)
    mdLGVDemand1_fac = factor_matrix(mdLGVDemand1, sample_mean, sd, noTAZ)
    mdHGVDemand1_fac = factor_matrix(mdHGVDemand1, sample_mean, sd, noTAZ)
    pmLGVDemand1_fac = factor_matrix(pmLGVDemand1, sample_mean, sd, noTAZ)
    pmHGVDemand1_fac = factor_matrix(pmHGVDemand1, sample_mean, sd, noTAZ)
    
    # putting the factored matrics in place to be assigned
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Am", amSovDemand1_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Am", amSovDemand2_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Am", amSovDemand3_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Am", amSovDemand4_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Md", mdSovDemand1_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Md", mdSovDemand2_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Md", mdSovDemand3_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Md", mdSovDemand4_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_1_Pm", pmSovDemand1_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_2_Pm", pmSovDemand2_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_3_Pm", pmSovDemand3_fac)
    util.set_matrix_numpy(eb, "mfSOV_drvtrp_VOT_4_Pm", pmSovDemand4_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Am", amHovDemand1_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Am", amHovDemand2_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Am", amHovDemand3_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Am", amHovDemand4_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Md", mdHovDemand1_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Md", mdHovDemand2_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Md", mdHovDemand3_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Md", mdHovDemand4_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_1_Pm", pmHovDemand1_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_2_Pm", pmHovDemand2_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_3_Pm", pmHovDemand3_fac)
    util.set_matrix_numpy(eb, "mfHOV_drvtrp_VOT_4_Pm", pmHovDemand4_fac)
    util.set_matrix_numpy(eb, "mflgvPceAm", amLGVDemand1_fac)
    util.set_matrix_numpy(eb, "mfhgvPceAm", amHGVDemand1_fac)
    util.set_matrix_numpy(eb, "mflgvPceMd", mdLGVDemand1_fac)
    util.set_matrix_numpy(eb, "mfhgvPceMd", mdHGVDemand1_fac)
    util.set_matrix_numpy(eb, "mflgvPcePm", pmLGVDemand1_fac)
    util.set_matrix_numpy(eb, "mfhgvPcePm", pmHGVDemand1_fac)
    
    auto_assign(amscen_i, mdscen_i, pmscen_i)
    
    # record run metadata
    md = {'day' : i,
            'sample_mean' : sample_mean,
            'days' : days,
            'daily_range': daily_range,
            'mean_days_share' : mean_days_share}
    md = pd.DataFrame(md, index=[0])
    
    # setup outputs to record results at GY level
    df = util.get_ijensem_df(eb, 'gy','gy')
    df['day'] = i

    df["amSovDemand1"] = amSovDemand1_fac.flatten()
    df["amSovDemand2"] = amSovDemand2_fac.flatten()
    df["amSovDemand3"] = amSovDemand3_fac.flatten()
    df["amSovDemand4"] = amSovDemand4_fac.flatten()
    df["mdSovDemand1"] = mdSovDemand1_fac.flatten()
    df["mdSovDemand2"] = mdSovDemand2_fac.flatten()
    df["mdSovDemand3"] = mdSovDemand3_fac.flatten()
    df["mdSovDemand4"] = mdSovDemand4_fac.flatten()
    df["pmSovDemand1"] = pmSovDemand1_fac.flatten()
    df["pmSovDemand2"] = pmSovDemand2_fac.flatten()
    df["pmSovDemand3"] = pmSovDemand3_fac.flatten()
    df["pmSovDemand4"] = pmSovDemand4_fac.flatten()
    df["amHovDemand1"] = amHovDemand1_fac.flatten()
    df["amHovDemand2"] = amHovDemand2_fac.flatten()
    df["amHovDemand3"] = amHovDemand3_fac.flatten()
    df["amHovDemand4"] = amHovDemand4_fac.flatten()
    df["mdHovDemand1"] = mdHovDemand1_fac.flatten()
    df["mdHovDemand2"] = mdHovDemand2_fac.flatten()
    df["mdHovDemand3"] = mdHovDemand3_fac.flatten()
    df["mdHovDemand4"] = mdHovDemand4_fac.flatten()
    df["pmHovDemand1"] = pmHovDemand1_fac.flatten()
    df["pmHovDemand2"] = pmHovDemand2_fac.flatten()
    df["pmHovDemand3"] = pmHovDemand3_fac.flatten()
    df["pmHovDemand4"] = pmHovDemand4_fac.flatten()
    df["amLGVDemand1"] = amLGVDemand1_fac.flatten()
    df["amHGVDemand1"] = amHGVDemand1_fac.flatten()
    df["mdLGVDemand1"] = mdLGVDemand1_fac.flatten()
    df["mdHGVDemand1"] = mdHGVDemand1_fac.flatten()
    df["pmLGVDemand1"] = pmLGVDemand1_fac.flatten()
    df["pmHGVDemand1"] = pmHGVDemand1_fac.flatten()


    df = pd.melt(df, id_vars = ['gy_i','gy_j', 'day'], var_name = 'timeModeVot', value_name='trips_fac')
    dfGy =  df.groupby(['gy_i','gy_j','day','timeModeVot'])
    dfGy = dfGy.sum().reset_index()
    dfTimeModeVot = dfGy['timeModeVot'].str.extract(r'(?P<peak>[a|m|p]{1}[m|d])(?P<mode>Sov|Hov|LGV|HGV)Demand(?P<votclass>\d)')
    dfGy = pd.concat([dfGy,dfTimeModeVot], axis=1)
    dfGy = dfGy[['gy_i','gy_j','day','peak','mode','votclass','trips_fac']]
    
    # pull auto network results
    dfAuto, dfTransit = data_export.networkExport(eb)
    dfAuto['day'] = i
    
    # write out data
    conn = util.get_db_byname(eb, 'trip_summaries.db')
    if i == 1:
        dfGy.to_sql(name='autoTripsGyVariable', con=conn, flavor='sqlite', index=False, if_exists='replace')
        md.to_sql(name='variableRunsMd', con=conn, flavor='sqlite', index=False, if_exists='replace')
        dfAuto.to_sql(name='netResultsVariable', con=conn, flavor='sqlite', index=False, if_exists='replace')
    else:
        dfGy.to_sql(name='autoTripsGyVariable', con=conn, flavor='sqlite', index=False, if_exists='append')
        md.to_sql(name='variableRunsMd', con=conn, flavor='sqlite', index=False, if_exists='append')
        dfAuto.to_sql(name='netResultsVariable', con=conn, flavor='sqlite', index=False, if_exists='append')
    conn.close()   